https://github.com/marketplace/models/azure-openai/gpt-4-1

In [1]:
import json
from pathlib import Path

import pandas as pd

from jinja2 import Template
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

from settings import GITHUB_TOKEN

# Defining the prompt

In [2]:
PROMPT = Path('../data/prompt.jinja').read_text()

In [3]:
# # ? - Test only
job_descriptions = pd.read_excel('../data/bmw_jobs.xlsx')
job_description = job_descriptions.loc[2, ('description')]

In [4]:
candidate_application = Path('/Users/caiopavesi/Downloads/kb_me.txt').read_text()

In [5]:
prompt = Template(PROMPT).render(job_description = job_description, candidate_application = candidate_application)

# Calling the ai

In [6]:
client = ChatCompletionsClient(
    endpoint = "https://models.github.ai/inference",
    credential = AzureKeyCredential(GITHUB_TOKEN),
)

In [7]:
response = client.complete(
    messages = [
        SystemMessage(
            "You are an Ai Agent of Talent Acquisition Specialist working as "
            "Consultant for a candidate to help him find jobs where he has the "
            "most chances of being called for an interview"
        ),
        UserMessage(prompt),
    ],
    temperature = 0.1,
    top_p = 1.0,
    model = "openai/gpt-4.1",
    response_format = "json_object",
)

In [8]:
result = json.loads(response.choices[0].message.content)

In [9]:
result

{'score': 92.5,
 'strengths': 'The candidate has direct, recent, and multi-faceted experience with BMW, including internships in production, logistics, and industrial engineering. He demonstrates strong technical skills in Python, Excel, Power Platform, and SQL, all relevant to software development. He has proven leadership and project management experience, both in industry and extracurricular activities (SAE racing team). His education aligns with the requirements, and he has completed relevant coursework in programming and databases. Multiple recommendations highlight his communication, teamwork, adaptability, and initiative. He also shows a strong cultural fit and motivation for BMW.',
 'weaknesses': 'Limited direct experience with C or C++ (though Python is present). German language skills are at a beginner level (A1), which may be a minor limitation for integration, though not a strict requirement for the role. No explicit mention of hands-on experience in automotive software dev